# Using GPUs in cloudknot

You can specify the number of GPUs for your cloudknot jobs using the `n_gpus` parameter of the `Knot` class.

In [1]:
import logging
import uuid
import cloudknot as ck

Set the logging level to print loging statements in the notebook

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Change this to logging.DEBUG if you want more verbose output

## Initialization

Create a function to output the number of GPUs. Remember that the import statements should be inside the function.

In [3]:
def get_n_gpus(job_idx):
    """Output the number of GPUs"""
    import torch

    return {
        "job_idx": job_idx,
        "device_count": torch.cuda.device_count()
    }

Create a `Knot` instance from the test function. Here we specify the instance type, selecting one of the [GPU instances](https://docs.aws.amazon.com/batch/latest/userguide/gpu-jobs.html). If you don't do this, your Batch jobs will be stuck in "RUNNABLE" status. Note also that we specify a `base_image` that has CUDA drivers.

In [4]:
knot = ck.Knot(
    name="count-gpu",
    base_image="pytorch/pytorch",
    func=get_n_gpus,
    n_gpus=2,
    instance_types=("p3.8xlarge",),
)

Let's see what happens if we don't specify the base image.

In [5]:
knot_without_base_image_specified = ck.Knot(
    name="count-gpu-no-base-image",
    func=get_n_gpus,
    n_gpus=2,
    instance_types=("p3.8xlarge",),
)

Now we dispatch the jobs on AWS Batch

In [6]:
result_futures = knot.map(list(range(5)))
result_futures_without_base_image = knot_without_base_image_specified.map(list(range(5)))

Let's check the results

In [10]:
result_futures.result()

[{'job_idx': 0, 'device_count': 2},
 {'job_idx': 1, 'device_count': 2},
 {'job_idx': 2, 'device_count': 2},
 {'job_idx': 3, 'device_count': 2},
 {'job_idx': 4, 'device_count': 2}]

Great! Each job got two GPUs as desired. Now let's see what happens when we don't specify a base image.

In [11]:
result_futures_without_base_image.result()

[{'job_idx': 0, 'device_count': 0},
 {'job_idx': 1, 'device_count': 0},
 {'job_idx': 2, 'device_count': 0},
 {'job_idx': 3, 'device_count': 0},
 {'job_idx': 4, 'device_count': 0}]

Boo! Okay, from now on, if you want GPUs for your Batch jobs, remember to specify `n_gpus`, an appropriate `instance_type`, and a `base_image` with the right device drivers.

Let's clean up our resources with the `clobber()` method.

In [ ]:
knot.clobber(clobber_pars=True, clobber_repo=True, clobber_image=True)
knot_without_base_image_specified.clobber(clobber_pars=True, clobber_repo=True, clobber_image=True)